In [33]:
from lightautoml.validation.base import TrainValidIterator as iters
from lightautoml.validation.base import HoldoutIterator as hiters


In [35]:
test = hiters(tr_data, te_data)

In [36]:
test[0]

TypeError: 'HoldoutIterator' object is not subscriptable

In [1]:
# Standard python libraries
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd

import cupy as cp
import cudf as cudf

from cuml import train_test_split

import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.gpu.tabular_gpu_presets import TabularAutoML_gpu
from lightautoml.tasks import Task
from lightautoml.report.gpu import ReportDeco

'pdf' extra dependecy package 'weasyprint' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'pdf' extra dependecy package 'weasyprint' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


In [2]:
import cudf
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

In [3]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 300
TARGET_NAME = 'FloodProbability'

In [4]:
DATASET_DIR = '/media/HDD2/Data/flood'
DATASET_NAMES = ['train.csv', 'test.csv', 'sample_submission.csv']
DATASET_FULLNAME = [os.path.join(DATASET_DIR, name) for name in DATASET_NAMES]

In [5]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [6]:
data = cudf.read_csv(os.path.join(DATASET_DIR, DATASET_NAMES[0]))
data.drop('id', axis=1, inplace=True)

In [7]:
data.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,5,8,5,8,6,4,4,3,3,4,...,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,...,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,...,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,...,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [8]:
for c in data.columns:
    print(c,len(data[c]))

MonsoonIntensity 1117957
TopographyDrainage 1117957
RiverManagement 1117957
Deforestation 1117957
Urbanization 1117957
ClimateChange 1117957
DamsQuality 1117957
Siltation 1117957
AgriculturalPractices 1117957
Encroachments 1117957
IneffectiveDisasterPreparedness 1117957
DrainageSystems 1117957
CoastalVulnerability 1117957
Landslides 1117957
Watersheds 1117957
DeterioratingInfrastructure 1117957
PopulationScore 1117957
WetlandLoss 1117957
InadequatePlanning 1117957
PoliticalFactors 1117957
FloodProbability 1117957


In [9]:
tr_data, te_data = train_test_split(data, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [10]:
desc = tr_data.nans_to_nulls().astype(object).describe(include='all')

/home/cvmi_jeyoung/anaconda3/envs/rapids/lib/python3.9/site-packages/cudf/core/series.py:3334: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


In [11]:
res = cudf.DataFrame(columns=['flg_manual', 'unique', 'unique_rate',\
                   'top_freq_values', 'raw_scores', 'binned_scores',\
                   'encoded_scores','freq_scores', 'nan_rate'],)

In [12]:
desc.loc['unique']

MonsoonIntensity                   17
TopographyDrainage                 19
RiverManagement                    17
Deforestation                      18
Urbanization                       18
ClimateChange                      18
DamsQuality                        17
Siltation                          17
AgriculturalPractices              17
Encroachments                      19
IneffectiveDisasterPreparedness    17
DrainageSystems                    18
CoastalVulnerability               18
Landslides                         17
Watersheds                         17
DeterioratingInfrastructure        18
PopulationScore                    19
WetlandLoss                        20
InadequatePlanning                 17
PoliticalFactors                   17
FloodProbability                   83
Name: unique, dtype: object

In [13]:
res['unique'] = desc.loc['unique']

In [14]:
res

,flg_manual,unique,unique_rate,top_freq_values,raw_scores,binned_scores,encoded_scores,freq_scores,nan_rate
MonsoonIntensity,<NA>,17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
TopographyDrainage,<NA>,19,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
RiverManagement,<NA>,17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Deforestation,<NA>,18,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Urbanization,<NA>,18,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
ClimateChange,<NA>,18,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DamsQuality,<NA>,17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Siltation,<NA>,17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
AgriculturalPractices,<NA>,17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Encroachments,<NA>,19,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [15]:
roles = {
    'target' : 'FloodProbability'
}

task = Task('reg',) #device='gpu')


automl = TabularAutoML_gpu(task=task,timeout=TIMEOUT, reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},)

In [16]:
from cudf.core.column.column import as_column

In [17]:
%%time 
oof_pred = automl.fit_predict(tr_data.to_pandas(), roles = roles, verbose = 1, valid_data = te_data.to_pandas())

[12:57:15] Stdout logging level is INFO.
[12:57:15] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:57:15] Task: reg

[12:57:15] Start automl preset with listed constraints:
[12:57:15] - time: 300.00 seconds
[12:57:15] - CPU: 4 cores
[12:57:15] - memory: 16 GB

[12:57:15] Train data shape: (894366, 21)


/home/cvmi_jeyoung/anaconda3/envs/rapids/lib/python3.9/site-packages/cudf/core/series.py:3334: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


[12:57:26] Feats was rejected during automatic roles guess: []
cudf reader: 11.053533008089289
[12:57:26] Layer 1 train process start. Time left 288.90 secs
[12:57:26] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...


TypeError: 'HoldoutIterator' object is not subscriptable